In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from astropy.time import Time

from adler.objectdata.AdlerPlanetoid import AdlerPlanetoid
from adler.objectdata.AdlerData import AdlerData
from adler.science.PhaseCurve import PhaseCurve
from adler.utilities.plotting_utilities import plot_errorbar, plot_phasecurve

In [ ]:
db_fname = "dp03_pop_sample.db"

In [ ]:
# phase_model = "HG12_Pen16"
# phase_model = "HG"
phase_model = "HG12"
pc_fname = "df_dp03_pop_sample_pc_{}.csv".format(phase_model)

In [ ]:
sample_fname = "df_ssoid_dp03_pop_sample.csv"
df_obj = pd.read_csv(sample_fname, index_col=0)
df_obj

In [ ]:
df_obj[df_obj["ssObjectId"] == 7299821094696955726]

In [ ]:
ad_params_list = []
error_list = []

for ssObjectId in df_obj["ssObjectId"]:
    print(ssObjectId)

    try:
        # load object data
        planetoid = AdlerPlanetoid.construct_from_SQL(
            ssObjectId,
            sql_filename=db_fname,
        )
    except:
        error_list.append(ssObjectId)
        continue

    for filt in planetoid.filter_list:
        #         print(filt)

        # get the observations
        obs = planetoid.observations_in_filter(filt)
        df_obs = pd.DataFrame(obs.__dict__)
        #         print(len(df_obs))

        # get cumulative number of observations with time
        nights = np.arange(
            int(np.amin(df_obs["midPointMjdTai"])) - 1, int(np.amax(df_obs["midPointMjdTai"])) + 1 + 1
        )
        values, base = np.histogram(df_obs["midPointMjdTai"], bins=nights)
        cumulative = np.cumsum(values)

        # select only nights when new observations come in
        data_mask = np.diff(cumulative) > 0
        #         data_nights = base[1:-1][data_mask]
        data_nights = base[2:][data_mask]
        N_data = cumulative[1:][data_mask]
        first_night = np.amin(data_nights)

        # Set initial values of fit
        H_start = np.amin(df_obs["reduced_mag"])
        phase_param_1_default = 0.62

        # for each data night find the phase curve fit
        for i, night in enumerate(data_nights):
            _df_obs = df_obs[(df_obs["midPointMjdTai"] <= night)]

            pc = PhaseCurve(
                H=H_start,
                phase_parameter_1=phase_param_1_default,
                model_name=phase_model,
            )

            # store the metadata
            ad_params = {}
            ad_params["ssObjectId"] = ssObjectId
            ad_params["filt"] = filt
            ad_params["data_night"] = night
            ad_params["delta_night"] = night - first_night
            ad_params["fit_number"] = i
            ad_params["phaseAngle_min"] = np.amin(_df_obs["phaseAngle"])
            ad_params["phaseAngle_range"] = np.ptp(_df_obs["phaseAngle"])
            ad_params["arc"] = np.ptp(_df_obs["midPointMjdTai"])
            ad_params["nobs"] = len(_df_obs)
            # TODO: more metadata? density of phaseangle coverage
            filt_bins = np.arange(
                0, int(np.amax(_df_obs["phaseAngle"])) + 2
            )  # +2 in order to span all observations
            hist, bin_edges = np.histogram(_df_obs["phaseAngle"], bins=filt_bins)
            ad_params["phaseAngle_sparsity"] = len(hist[hist == 0]) / len(
                hist
            )  # fractional number of 1 degree phase angle bins without observations
            # TODO: think about best way to do "density"
            #             ad_params["phaseAngle_density"] = np.mean(hist) # mean number of observations per 1 degree phase angle bin
            try:
                ad_params["phaseAngle_density"] = (
                    len(_df_obs) / ad_params["phaseAngle_range"]
                )  # mean number of observations per unit phase angle
            except:
                ad_params["phaseAngle_density"] = np.nan

            # if there are too few observations, don't fit
            if len(_df_obs) < len(pc.model_function.parameters):
                ad_params_list.append(ad_params)
                continue

            # do a phase model fit to the past data
            try:
                pc_fit = pc.FitModel(
                    np.radians(np.array(_df_obs["phaseAngle"])),
                    np.array(_df_obs["reduced_mag"]),
                    np.array(_df_obs["magErr"]),
                )
            # if the fit fails, continue
            except:
                print("Fitting error, {}".format(ssObjectId))
                ad_params_list.append(ad_params)
                continue

            # Store the fitted values, and metadata, in an AdlerData object
            pc_fit = pc.InitModelSbpy(pc_fit)
            for x in [x for x in pc_fit.__dict__ if x != "model_function"]:
                ad_params[x] = pc_fit.__dict__[x]
            ad_params["modelFitMjd"] = Time.now().mjd

            # TODO: calculate differences in fitted parameters? Use SSObject values?
            # delta H
            # delta phase_parameter_1

            # Update initial values using this fit
            H_start = ad_params["H"]
            phase_param_1_default = ad_params["phase_parameter_1"]

            # add this fit to the list
            ad_params_list.append(ad_params)

#     break

In [ ]:
df_ad = pd.DataFrame(ad_params_list)

In [ ]:
df_ad

In [ ]:
df_ad.to_csv(pc_fname)

In [ ]:
# compare the incremental fits to the SSObject values

In [ ]:
# df_test = df_ad.groupby(by=["ssObjectId","filt","model_name"]).max().reset_index()

In [ ]:
df_ad[df_ad["phaseAngle_sparsity"] == 0]

In [ ]:
df_ad[df_ad["phaseAngle_range"] == 0]

In [ ]:
df_ad[(df_ad["phaseAngle_sparsity"] == 0) & (df_ad["ssObjectId"] == -8884019454025057114)]

In [ ]:
# for i,ssObjectId in enumerate(df_obj["ssObjectId"].sample(10)):
for i, ssObjectId in enumerate([-8884019454025057114]):

    print(ssObjectId)

    # load object data
    planetoid = AdlerPlanetoid.construct_from_SQL(
        ssObjectId,
        sql_filename=db_fname,
    )

    fig = plot_errorbar(planetoid, filt_list=planetoid.filter_list)

    filt_hists = []

    for j, filt in enumerate(planetoid.filter_list):

        df_fit = (
            df_ad[(df_ad["ssObjectId"] == ssObjectId) & (df_ad["filt"] == filt)]
            .sort_values("data_night")
            .iloc[-1]
        )

        pc = PhaseCurve(
            H=df_fit["H"],
            phase_parameter_1=df_fit["phase_parameter_1"],
            model_name=df_fit["model_name"],
        )

        obs = planetoid.observations_in_filter(filt)
        df_obs = pd.DataFrame(obs.__dict__)

        x = np.radians(np.linspace(0, np.amax(df_obs["phaseAngle"])))
        y = pc.ReducedMag(x)
        ax1 = fig.gca()
        ax1.plot(np.degrees(x), y, color="C{}".format(j), label=filt)

        # calculate density of coverage
        filt_bins = np.arange(0, int(np.amax(df_obs["phaseAngle"])) + 2)
        #         y = pc.ReducedMag(np.radians(filt_bins))
        #         ax1 = fig.gca()
        #         ax1.scatter(filt_bins, y, color="k")
        hist, bin_edges = np.histogram(df_obs["phaseAngle"], bins=filt_bins)
        filt_hists.append(hist)
        print(filt)
        print("mean={}, std={}".format(np.mean(hist), np.std(hist)))
        print(len(df_obs) / np.ptp(df_obs["phaseAngle"]))
        #         print(np.median(hist))
        print("sparsity={}".format(len(hist[hist == 0]) / len(hist)))

    ax1.legend()
    plt.show()

#     fig = plt.figure()
#     gs = gridspec.GridSpec(1,1)
#     ax1 = plt.subplot(gs[0,0])


#     break

#     if i>=3-1:
#         break

In [ ]:
df_obs

In [ ]:
hist, bin_edges = np.histogram(df_obs["phaseAngle"], bins=filt_bins)

In [ ]:
len(hist), len(bin_edges)

In [ ]:
hist